In [1]:
import hashlib
import pandas as pd
import geopandas as gpd
from shapely.wkt import dumps

In [2]:
df = gpd.read_file('../.data/census blocks')
df.head()

,OBJECTID,STATE,COUNTY,TRACT,TRACT_NO,BG,TRBG,FIPS,POP10,DU10,...,BLACK,AIAN,ASIAN,NHPI,OTHER_RACE,POP_2_RACE,HISPANIC,Shape_STAr,Shape_STLe,geometry
0,1,41,051,005500,55.00,1,0055001,410510055001,1122,887,...,54,13,56,3,14,42,63,2.789209e+06,8603.607511,"POLYGON ((7639530.427493438 680741.863188982, ..."
1,2,41,005,022800,228.00,1,0228001,410050228001,908,726,...,3,3,11,2,13,7,35,5.544203e+06,12154.191446,"POLYGON ((7618198.371391073 600580.9750656188,..."
2,3,41,005,022800,228.00,3,0228003,410050228003,843,330,...,2,9,12,0,21,10,49,2.096738e+08,104189.416208,"POLYGON ((7623979.813648298 597306.4156824201,..."
3,4,41,005,022906,229.06,1,0229061,410050229061,1577,805,...,2,12,10,3,102,13,140,2.367821e+07,24077.575007,"POLYGON ((7638793.470144361 583141.7887139171,..."
4,5,41,005,022800,228.00,4,0228004,410050228004,662,230,...,2,7,10,0,37,17,57,1.685747e+08,80724.100634,"POLYGON ((7633390.819881886 595356.693241477, ..."


In [3]:
df.crs

{'init': 'epsg:2913'}

In [4]:
df = df[['STATE', 'COUNTY', 'TRACT', 'BG', 'FIPS', 'geometry']].rename(index=str, columns={
    'STATE': 'state',
    'COUNTY': 'county',
    'TRACT': 'tract',
    'BG': 'bg',
    'FIPS': 'fips'
})

In [5]:
df = df.to_crs(epsg=4326)

df['wkt'] = df.geometry.map(dumps)
df['hash'] = df.wkt.map(lambda x: hashlib.md5(
    x.encode('utf-8')).hexdigest())

df['center'] = df.geometry.map(lambda x: x.centroid)
df['center_x'] = df.center.map(lambda x: x.x)
df['center_y'] = df.center.map(lambda x: x.y)

df = df.to_crs(epsg=3857)
df['area'] = df.geometry.map(lambda x: x.area)

In [6]:
df.head()

,state,county,tract,bg,fips,geometry,wkt,hash,center,center_x,center_y,area
0,41,051,005500,1,410510055001,POLYGON ((-13658329.99021978 5702635.838281421...,POLYGON ((-122.6948658576246345 45.51292564063...,d2915da83140d134ea11868742e37920,POINT (-122.6912068656927 45.51315091200215),-122.691207,45.513151,5.277385e+05
1,41,005,022800,1,410050228001,"POLYGON ((-13666612.35532421 5667534.45721275,...",POLYGON ((-122.7692676092442809 45.29153114119...,b9552bc92a37779cbf29704ec224b50e,POINT (-122.7654080065694 45.28745343510226),-122.765408,45.287453,1.040744e+06
2,41,005,022800,3,410050228003,POLYGON ((-13664073.03937506 5666184.015581907...,POLYGON ((-122.7464565459609958 45.28299618624...,76f9aae49e336e48f50c2316a71509d6,POINT (-122.7640442110357 45.27897182168579),-122.764044,45.278972,3.934787e+07
3,41,005,022906,1,410050229061,POLYGON ((-13657502.49303695 5660217.894332951...,POLYGON ((-122.6874323239555764 45.24527420679...,cd62747fdefd0d04a3b2d904627a6c5c,POINT (-122.6984451663146 45.24932517578785),-122.698445,45.249325,4.438899e+06
4,41,005,022800,4,410050228004,POLYGON ((-13659981.15364409 5665450.894052321...,POLYGON ((-122.7096985110309646 45.27836223141...,1abd592b27e3511580ea33cfd5c96d9a,POINT (-122.7252279436671 45.25254554896382),-122.725228,45.252546,3.160586e+07


In [7]:
df[[
    'hash',
    'state',
    'county',
    'tract',
    'bg',
    'fips',
    'center_x',
    'center_y',
    'area',
    'wkt'
]].to_csv('../.data/census_block_groups.csv', index_label='key')

In [8]:
df[[
    'hash',
    'state',
    'county',
    'tract',
    'bg',
    'fips',
    'center_x',
    'center_y',
    'area'
]].to_csv('../.data/census_block_groups_etl.csv', index_label='key')